In [1]:
import pandas as pd
import numpy as np
import os
from env import host, user, password
import acquire

In [2]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import explore

plt.rcParams['figure.figsize'] = (4, 2)

In [3]:
df = acquire.get_telco_data()
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


def clean_telco(df):
    df.total_charges.replace(to_replace = {" ":"0"}, inplace = True)
    df.total_charges = df.total_charges.astype("float")
    df["auto_pay"] = df.payment_type.str.contains("auto")
    df["auto_pay"] = df.auto_pay.replace(to_replace = [True,False],value = [1,0])
    dummy_df = pd.get_dummies(df[['gender', 'payment_type',"internet_service_type","contract_type"]])
    
    df["partner"] = df.partner.replace(to_replace = ["Yes","No"],value = [1,0])
    df["dependents"] = df.dependents.replace(to_replace = ["Yes","No"],value = [1,0])
    df["churn"] = df.churn.replace(to_replace = ["Yes","No"],value = [1,0])
    df["multiple_lines"] = df.paperless_billing.replace(to_replace = ["Yes","No","No phone service"],value = [1,0,0])
    df["paperless_billing"] = df.paperless_billing.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["tech_support"] = df.tech_support.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["device_protection"] = df.device_protection.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["online_backup"] = df.online_backup.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["online_security"] = df.online_security.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["streaming_tv"] = df.streaming_tv.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["streaming_movies"] = df.streaming_movies.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["phone_service"] = df.phone_service.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
    df["multiple_lines"] = df.multiple_lines.replace(to_replace = ["Yes","No","No phone service"],value = [1,0,0])
    df.drop(columns = ["customer_id",
                       "gender",
                       "payment_type",
                       "internet_service_type",
                       "contract_type",
                   'internet_service_type_id',
                   "contract_type_id",
                   "payment_type_id",
                  "contract_type_id.1",
                   "internet_service_type_id.1",
                   "payment_type_id.1",
                   "Unnamed: 0"],inplace = True)
    return pd.concat([df, dummy_df], axis=1)

In [4]:
df.isna().sum()

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [6]:
# found 11 customers who have blank total charges, are under contract, but have not churned. However their tenure is 0 so we know they are new customers. Therefore, I will replace the blanks with 0's. 
df.total_charges.replace(to_replace = {" ":"0"}, inplace = True)
df.total_charges = df.total_charges.astype("float")
df.total_charges.describe()

count    7043.000000
mean     2279.734304
std      2266.794470
min         0.000000
25%       398.550000
50%      1394.550000
75%      3786.600000
max      8684.800000
Name: total_charges, dtype: float64

In [7]:
df.head(2)

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.90,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)


In [8]:
pd.set_option("display.max_columns", None)

In [9]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,90.45,5957.90,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,No phone service,Yes,No,No,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,No phone service,Yes,Yes,Yes,Yes,No,No,No,45.05,2560.10,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,Yes,No,Yes,Yes,No,No,Yes,39.40,825.40,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,85.15,6316.20,No,Two year,DSL,Bank transfer (automatic)


In [10]:
df["auto_pay"] = df.payment_type.str.contains("auto")
df["auto_pay"] = df.auto_pay.replace(to_replace = [True,False],value = [1,0])
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,auto_pay
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,90.45,5957.90,No,Two year,DSL,Mailed check,0
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,No phone service,Yes,No,No,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic),1
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,No phone service,Yes,Yes,Yes,Yes,No,No,No,45.05,2560.10,No,Two year,DSL,Bank transfer (automatic),1
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,Yes,No,Yes,Yes,No,No,Yes,39.40,825.40,No,Two year,DSL,Credit card (automatic),1
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,85.15,6316.20,No,Two year,DSL,Bank transfer (automatic),1


In [11]:
dummy_df = pd.get_dummies(df[['gender',"internet_service_type","contract_type"]])
dummy_df.drop(columns=['gender_Female', 'internet_service_type_None', 'contract_type_Month-to-month'], inplace=True)


In [12]:
dummy_df.head(1)

,gender_Male,internet_service_type_DSL,internet_service_type_Fiber optic,contract_type_One year,contract_type_Two year
0,0,1,0,0,1


In [13]:
df = pd.concat([df, dummy_df], axis=1)
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,auto_pay,gender_Male,internet_service_type_DSL,internet_service_type_Fiber optic,contract_type_One year,contract_type_Two year
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,90.45,5957.90,No,Two year,DSL,Mailed check,0,0,1,0,0,1
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,No phone service,Yes,No,No,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic),1,1,1,0,0,1
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,No phone service,Yes,Yes,Yes,Yes,No,No,No,45.05,2560.10,No,Two year,DSL,Bank transfer (automatic),1,0,1,0,0,1
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,Yes,No,Yes,Yes,No,No,Yes,39.40,825.40,No,Two year,DSL,Credit card (automatic),1,1,1,0,0,1
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,85.15,6316.20,No,Two year,DSL,Bank transfer (automatic),1,1,1,0,0,1


In [14]:
df["partner"] = df.partner.replace(to_replace = ["Yes","No"],value = [1,0])
df["dependents"] = df.dependents.replace(to_replace = ["Yes","No"],value = [1,0])
df["phone_service"] = df.phone_service.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
df["multiple_lines"] = df.multiple_lines.replace(to_replace = ["Yes","No","No phone service"],value = [1,0,0])
df["online_security"] = df.online_security.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
df["online_backup"] = df.online_backup.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
df["device_protection"] = df.device_protection.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
df["tech_support"] = df.tech_support.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
df["streaming_tv"] = df.streaming_tv.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
df["streaming_movies"] = df.streaming_movies.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
df["paperless_billing"] = df.paperless_billing.replace(to_replace = ["Yes","No","No internet service"],value = [1,0,0])
df["churn"] = df.churn.replace(to_replace = ["Yes","No"],value = [1,0])

df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,auto_pay,gender_Male,internet_service_type_DSL,internet_service_type_Fiber optic,contract_type_One year,contract_type_Two year
0,2,1,3,0016-QLJIS,Female,0,1,1,65,1,1,1,1,1,1,1,1,1,90.45,5957.90,0,Two year,DSL,Mailed check,0,0,1,0,0,1
1,4,1,3,0017-DINOC,Male,0,0,0,54,0,0,1,0,0,1,1,0,0,45.20,2460.55,0,Two year,DSL,Credit card (automatic),1,1,1,0,0,1
2,3,1,3,0019-GFNTW,Female,0,0,0,56,0,0,1,1,1,1,0,0,0,45.05,2560.10,0,Two year,DSL,Bank transfer (automatic),1,0,1,0,0,1
3,4,1,3,0056-EPFBG,Male,0,1,1,20,0,0,1,0,1,1,0,0,1,39.40,825.40,0,Two year,DSL,Credit card (automatic),1,1,1,0,0,1
4,3,1,3,0078-XZMHT,Male,0,1,0,72,1,1,0,1,1,1,1,1,1,85.15,6316.20,0,Two year,DSL,Bank transfer (automatic),1,1,1,0,0,1


In [16]:
df["add_ons"] =  ((df.online_security + df.online_backup + df.device_protection + df.tech_support + df.streaming_tv + df.streaming_movies) > 0)
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,auto_pay,gender_Male,internet_service_type_DSL,internet_service_type_Fiber optic,contract_type_One year,contract_type_Two year,add_ons
0,2,1,3,0016-QLJIS,Female,0,1,1,65,1,1,1,1,1,1,1,1,1,90.45,5957.90,0,Two year,DSL,Mailed check,0,0,1,0,0,1,True
1,4,1,3,0017-DINOC,Male,0,0,0,54,0,0,1,0,0,1,1,0,0,45.20,2460.55,0,Two year,DSL,Credit card (automatic),1,1,1,0,0,1,True
2,3,1,3,0019-GFNTW,Female,0,0,0,56,0,0,1,1,1,1,0,0,0,45.05,2560.10,0,Two year,DSL,Bank transfer (automatic),1,0,1,0,0,1,True
3,4,1,3,0056-EPFBG,Male,0,1,1,20,0,0,1,0,1,1,0,0,1,39.40,825.40,0,Two year,DSL,Credit card (automatic),1,1,1,0,0,1,True
4,3,1,3,0078-XZMHT,Male,0,1,0,72,1,1,0,1,1,1,1,1,1,85.15,6316.20,0,Two year,DSL,Bank transfer (automatic),1,1,1,0,0,1,True


In [18]:
df["add_ons"] = df.add_ons.replace(to_replace = [True, False],value = [1,0])
df.add_ons

0       1
1       1
2       1
3       1
4       1
       ..
7038    0
7039    0
7040    0
7041    0
7042    0
Name: add_ons, Length: 7043, dtype: int64

In [20]:
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,auto_pay,gender_Male,internet_service_type_DSL,internet_service_type_Fiber optic,contract_type_One year,contract_type_Two year,add_ons
0,2,1,3,0016-QLJIS,Female,0,1,1,65,1,1,1,1,1,1,1,1,1,90.45,5957.90,0,Two year,DSL,Mailed check,0,0,1,0,0,1,1
1,4,1,3,0017-DINOC,Male,0,0,0,54,0,0,1,0,0,1,1,0,0,45.20,2460.55,0,Two year,DSL,Credit card (automatic),1,1,1,0,0,1,1
2,3,1,3,0019-GFNTW,Female,0,0,0,56,0,0,1,1,1,1,0,0,0,45.05,2560.10,0,Two year,DSL,Bank transfer (automatic),1,0,1,0,0,1,1
3,4,1,3,0056-EPFBG,Male,0,1,1,20,0,0,1,0,1,1,0,0,1,39.40,825.40,0,Two year,DSL,Credit card (automatic),1,1,1,0,0,1,1
4,3,1,3,0078-XZMHT,Male,0,1,0,72,1,1,0,1,1,1,1,1,1,85.15,6316.20,0,Two year,DSL,Bank transfer (automatic),1,1,1,0,0,1,1


In [22]:
df1 = df.drop(columns= ['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id', 'gender', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'contract_type', 'internet_service_type', 'payment_type'])
df1.head()                        

,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,paperless_billing,monthly_charges,total_charges,churn,auto_pay,gender_Male,internet_service_type_DSL,internet_service_type_Fiber optic,contract_type_One year,contract_type_Two year,add_ons
0,0,1,1,65,1,1,1,90.45,5957.90,0,0,0,1,0,0,1,1
1,0,0,0,54,0,0,0,45.20,2460.55,0,1,1,1,0,0,1,1
2,0,0,0,56,0,0,0,45.05,2560.10,0,1,0,1,0,0,1,1
3,0,1,1,20,0,0,1,39.40,825.40,0,1,1,1,0,0,1,1
4,0,1,0,72,1,1,1,85.15,6316.20,0,1,1,1,0,0,1,1


0       65.869541
1       54.436947
2       56.827969
3       20.949239
4       74.177334
          ...    
7038     1.000000
7039    18.469849
7040     6.576142
7041     1.000000
7042     4.081146
Length: 7043, dtype: float64